<a href="https://colab.research.google.com/github/av-gav/itmo-nlp/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia
!pip install tatoebatools
from tatoebatools import tatoeba
import wikipedia
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=902a898b925fa49883f3ab7fad1455ee363734436882712764aac3ad88f5d785
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.23-cp310-cp310-linux_x86_64.whl size=1554857 sha256=4d4eb88349753c8d02470b07d9c46f5246643045bd47a7ac99819aa29186256c
  Stored in directory: /root/.cache/pip/wheels/b1/83/74/6276352424e42f57b3878fb724f70cbd684af64bd4f5f5a04f
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    

В качестве источников данных будем использовать Википедию, сайт Tatoeba и переводы Декларации прав человека с сайта unicode.org. Так как метки языков на этих сайтах разные, нужна таблица трансляции.

In [35]:
# List of languages with cyrillic script, for which there are texts in Wikipedia, Tatoeba and UDHR

lang_str = '''Abkhazian	ab	abk	abk
Adyghe	ady	ady	ady
Altai	alt	alt	alt
Avaric	av	ava	
Bashkir	ba	bak	
Belarusian	be	bel	bel
Bulgarian	bg	bul	bul
Buriat	bxr	bua	
Chechen	ce	che	
Chuvash	cv	chv	
Eastern Mari	mhr	mhr	
Erzya	myv	myv	
Ingush	inh	inh	
Kabardian	kbd	kbd	kbd
Kalmyk	xal	xal	
Karachay-Balkar	krc	krc	
Kazakh	kk	kaz	kaz
Komi	kv	kpv	
Komi-Permyak	koi	koi	koi
Kyrgyz	ky	kir	kir
Lak	lbe	lbe	
Lezghian	lez	lez	
Macedonian	mk	mkd	mkd
Moksha	mdf	mdf	
Mongolian	mn	mon	khk
Ossetic	os	oss	oss
Russian	ru	rus	rus
Rusyn	rue	rue	
Serbian	sr	srp	srp_cyrl
Tajik	tg	tgk	tgk
Tatar	tt	tat	tat
Tuvinian	tyv	tyv	tyv
Udmurt	udm	udm	
Ukrainian	uk	ukr	ukr
Western Mari	mrj	mrj	
Yakut	sah	sah	sah'''

lang_data = {y[0]:{"wiki":y[1], "tato":y[2], "udhr":y[3], "label": i} for i,y in enumerate([x.split('\t') for x in lang_str.split('\n')])}
lang_data

{'Abkhazian': {'wiki': 'ab', 'tato': 'abk', 'udhr': 'abk', 'label': 0},
 'Adyghe': {'wiki': 'ady', 'tato': 'ady', 'udhr': 'ady', 'label': 1},
 'Altai': {'wiki': 'alt', 'tato': 'alt', 'udhr': 'alt', 'label': 2},
 'Avaric': {'wiki': 'av', 'tato': 'ava', 'udhr': '', 'label': 3},
 'Bashkir': {'wiki': 'ba', 'tato': 'bak', 'udhr': '', 'label': 4},
 'Belarusian': {'wiki': 'be', 'tato': 'bel', 'udhr': 'bel', 'label': 5},
 'Bulgarian': {'wiki': 'bg', 'tato': 'bul', 'udhr': 'bul', 'label': 6},
 'Buriat': {'wiki': 'bxr', 'tato': 'bua', 'udhr': '', 'label': 7},
 'Chechen': {'wiki': 'ce', 'tato': 'che', 'udhr': '', 'label': 8},
 'Chuvash': {'wiki': 'cv', 'tato': 'chv', 'udhr': '', 'label': 9},
 'Eastern Mari': {'wiki': 'mhr', 'tato': 'mhr', 'udhr': '', 'label': 10},
 'Erzya': {'wiki': 'myv', 'tato': 'myv', 'udhr': '', 'label': 11},
 'Ingush': {'wiki': 'inh', 'tato': 'inh', 'udhr': '', 'label': 12},
 'Kabardian': {'wiki': 'kbd', 'tato': 'kbd', 'udhr': 'kbd', 'label': 13},
 'Kalmyk': {'wiki': 'xal', 

Скрипт для получения данных с Википедии. Отбираем названия страниц, для которых созданы статьи на большом количестве языков. Загружаем каждую статью на английском, и ищем в ссылках на другие языки те языки, которые находятся в нашем списке. Эти ссылки запоминаем.

In [47]:
# get links to receive data from Wikipedia

hrefs = {}
wiki_langs = [v['wiki'] for v in lang_data.values()]
wiki_pages = ['Che Guevara', 'Sport', 'Computer', 'Europe', 'China', 'Year', 'History', 'Metal', 'Japan', 'City', 'Music', 'Rome', 'Earth', 'Milk', 'Mathematics', 'Air', 'South America', 'Dog', 'Bulgaria', 'Sea', 'Money', 'River', 'Egypt', 'Water', 'Tiger', 'Oxygen', 'Rain', 'United Nations', 'Science', 'Star', 'Geography', 'Food', 'Nicaragua', 'Leonardo da Vinci', 'Dmitry Gulia', 'Circassians', 'Association football', 'Italy', 'Heart', 'Time', 'Bible', 'Atom', 'Art', 'Sun', 'Human', 'Cyprus', 'New York City', 'Culture', 'Moon', 'Wikipedia', 'North America', 'Islam', 'Color', 'Animal', 'Ocean', 'Fire', 'Gold', 'Planet', 'Bird', 'Cat', 'Christianity', 'Paris', 'Belarus', 'Eye', 'Month', 'Africa', 'Plant', 'Physics', 'Tree', 'Mountain', 'Fish', 'Number', 'Finland', 'Bread', 'India', 'Language', 'Continent', 'Ukraine', 'Education', 'Russia', 'Internet', 'Iron', 'Asia', '20th century', 'Religion']
for page in wiki_pages:
  print(page, end=' ')
  html = requests.get(f'https://en.wikipedia.org/wiki/{page}')
  soup = BeautifulSoup(html.text, 'html.parser')
  links = soup.select("#p-lang-btn a.interlanguage-link-target")
  for a in links:
    lang = a['hreflang']
    if lang in wiki_langs:
      if lang not in hrefs:
        hrefs[lang] = []
      hrefs[lang] += [requests.utils.unquote(a['href'])]
      print(".", end='')
      time.sleep(.05)
  print()

Che Guevara ......................
Sport .......................
Computer ........................
Europe ..................................
China ..................................
Year ............................
History ...........................
Metal .................
Japan ....................................
City .............................
Music ......................
Rome .............................
Earth ...................................
Milk ........................
Mathematics ........................
Air ..............
South America ............................
Dog ................................
Bulgaria ..................................
Sea ..............................
Money ..........................
River ..........................
Egypt ...............................
Water .........................
Tiger .......................
Oxygen ..........................
Rain .......................
United Nations .............................
Science .............

Теперь скачиваем сами статьи по ссылкам. Если для какого-то из языков текстов получено мало. то начинаем рандомный поиск статей, пока не наберем достаточное количество текстов.

In [69]:
cyrillic = "".join([chr(x) for x in range(0x0400, 0x0460)] + [chr(x) for x in range(0x048A, 0x0500)])

def check_cyrillic(word):   # return true if all symbols in the string are cyrillic or non alphabetic
  for ch in word:
    if ch not in cyrillic and ch.isalpha():   # alphabet chars not from cyriliic table are not allowed
      return False
  return True

def handle_link(link):
  article = requests.get(link)
  soup = BeautifulSoup(article.text, 'html.parser')
  contents = soup.select("p")
  texts = []
  for p in contents:
    text = p.get_text()
    t = re.sub(r' \([^)]*\)| \[[^\]]*\]| «[^»]*»', '', text)   # remove text in brackets
    t = re.sub(r'\w+', '', t, flags = re.ASCII)   # remove digits and ASCII letters
    t = re.sub(r"\W[^\w\s’]+", '', t)
    t = re.sub(r'\s\w\W\s', ' ', ' '+ t +' ')
    t = re.sub(r'\s+', ' ', t.strip())   # replace all multiple blank charachters with single space
    if len(t) < 32:
      continue
    if check_cyrillic(t):
      texts.append(t)
  return texts

dataset = []
for k,v in lang_data.items():
  lang = v['wiki']
  links = hrefs[lang]
  wikipedia.set_lang(lang)
  wikipedia.set_rate_limiting(True)
  texts = []
  print(lang, end=' ')
  time.sleep(.05)
  for link in links:
    texts.extend(handle_link(link))
    print('=', end='')
    if len(texts) > 1024:
      break
  print(f" {len(texts)} ", end='')

  while len(texts) < 40:
    for page in wikipedia.random(10):
      try:
        p = wikipedia.page(page)
        texts.extend(handle_link(p.url))
      except wikipedia.exceptions.DisambiguationError:
        pass
    print(f"{ len(texts)} ", end='')
  for t in set(texts):      # remove duplicates
    dataset.append((t, k))
  print("@")

wiki = pd.DataFrame.from_records(dataset, columns=['text', 'lang'])
wiki.to_csv('wiki.csv', sep='\t')

ab ========================================== 37 47 @
ady =============================== 82 @
alt =========== 380 @
av ================================================ 198 @
ba ================================== 1043 @
be ================================= 1031 @
bg ====================== 1053 @
bxr ============================================================================== 574 @
ce ==================================================================== 806 @
cv ================================================================================= 73 @
mhr =========================================================== 252 @
myv ============================================================== 215 @
inh ================================================================= 146 @
kbd ============================= 245 @
xal ======================================== 122 @
krc =============================================== 431 @
kk =============================== 1037 @
kv =================================

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


3 3 8 11 12 13 13 14 14 14 14 14 14 15 15 15 17 17 18 19 22 23 25 26 27 28 28 28 29 29 30 31 32 33 34 37 38 38 40 @
ru ============= 1117 @
rue =============================================================================== 202 @
sr ===================== 1033 @
tg ============================================================================== 537 @
tt ============================================================================ 1057 @
tyv ========================= 137 @
udm ================================================= 93 @
uk ================== 1090 @
mrj ================================= 62 @
sah ================================================================================== 490 @


Теперь скачииваем предложения с сайта Tatoeba. Слишком короткие предложения не сохраняем. Также если количество текстов для данного языка больше 2048, прекращаем скачивание, чтобы не было дисбаланса классов.

In [73]:
# get texts from Tatoeba

for k,v in lang_data.items():
  lang = v['tato']
  count = 0
  for sent in tatoeba.sentences_detailed(lang):
    if len(sent.text) < 48:
      continue
    dataset.append([sent.text, k])
    count += 1
    if count > 2048:
      break


Ну и добавляем тексты Декларации о правах человека. Хотя все тексты есть в plaintext-формате, в данном случае удобнее получать в HTML, чтобы проще было убрать заголовки (они идентичные, для нашей задачи будут лишними)

In [71]:
# get texts from UDHR
for k,v in lang_data.items():
  lang = v['udhr']
  if not lang:
    continue
  url = f'https://www.unicode.org/udhr/d/udhr_{lang}.html'
  article = requests.get(url)
  soup = BeautifulSoup(article.text, 'html.parser')
  contents = soup.select("div p")
  for p in contents:
    text = p.get_text()
    t = re.sub(r'\w+', '', text, flags = re.ASCII)   # remove digits and ASCII letters
    t = re.sub(r"\W[^\w\s’]+", '', t)
    t = re.sub(r'\s\w\W\s', ' ', ' '+ t +' ')
    t = re.sub(r'\s+', ' ', t.strip())   # replace all multiple blank charachters with single space
    if t:
      dataset.append([t, k])

In [74]:
len(dataset)

30056

После нескольких попыток стало видно, что мешает пунктуация, так что на этом этапе имеет смысл привести все слова к строчным буквам и вообще убрать все небуквенные символы. 

In [75]:
dataset_cleared = []
for item in dataset:
  text, lang = item[0], item[1]
  ts = re.split(r'[\W\d]+', text.lower()) 
  dataset_cleared.append([' '.join([w for w in ts if check_cyrillic(w)]).strip(), lang])





In [76]:
df = pd.DataFrame.from_records(dataset_cleared, columns=['text', 'lang'])

In [77]:
df.to_csv('dataset_cleared2.csv', sep='\t')

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

Строим базовую модель с Count-векторайзером.

In [127]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['lang'], random_state=2023)

vec = CountVectorizer(ngram_range=(1, 2), analyzer='char_wb')
X_train_vectorized = vec.fit_transform(X_train)
X_test_vectorized = vec.transform(X_test)

In [79]:
vec.get_feature_names_out()

array([' в', ' г', ' е', ..., 'ӹм', 'ӹн', 'ӹш'], dtype=object)

Используем логистическую регрессию для модели.

In [128]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
clf = LogisticRegression(max_iter=500).fit(X_train_vectorized, y_train)
preds = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                 precision    recall  f1-score   support

      Abkhazian       0.92      0.96      0.94        23
         Adyghe       1.00      0.92      0.96        37
          Altai       1.00      0.97      0.98        96
         Avaric       0.95      0.95      0.95        41
        Bashkir       1.00      0.96      0.98       277
     Belarusian       0.99      0.99      0.99       627
      Bulgarian       0.97      0.98      0.97       527
         Buriat       0.92      0.93      0.92       141
        Chechen       0.98      0.95      0.97       174
        Chuvash       0.87      0.87      0.87        38
   Eastern Mari       0.94      0.95      0.94       257
          Erzya       0.96      0.92      0.94        53
         Ingush       0.74      0.82      0.78        34
      Kabardian       0.97      0.97      0.97        77
         Kalmyk       0.94      0.76      0.84        41
Karachay-Balkar       0.87      0.87      0.87       102
         Kazakh       0.99    

Получились вполне неплохие результаты для начала.

In [ ]:
pd.DataFrame(X_train_vectorized.toarray(), columns=vec.get_feature_names_out()).head()

,"""",-,.,«,²,³,¹,е,ж,з,...,’е,’я,“,“,”,”,•,•,…,…
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,11,20,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Посмотрим, какие ошибки совершала модель при предсказании. Видно, что в основном это либо короткие фразы, либо родственные языки. 

In [85]:
res = pd.DataFrame({'txt':X_test, 'lng':y_test, 'pred':preds})
res.loc[res['lng'] != res['pred']]

,txt,lng,pred
22656,ала куку муралта сулыкаҥын сӧрен шӱдӧ ийым ку ...,Eastern Mari,Yakut
24574,јас сум оној којшто мора да изврши најмногу ра...,Macedonian,Serbian
10821,хүн төрөлхтний түүхэнд олон шашин буй болж өөр...,Mongolian,Buriat
22417,доколкото можем да преценим том не е бил замес...,Bulgarian,Macedonian
19018,,Tatar,Serbian
...,...,...,...
4837,юань лела кехат ахчана кепара эчиган ахчана ке...,Chechen,Ingush
12766,у условима глобалне равнотеже на парцијалним р...,Serbian,Macedonian
21975,световната икономика е изложена на сериозна оп...,Bulgarian,Serbian
10381,далайн нийт масс тонн буюу дэлхийн нийт массын...,Mongolian,Buriat


In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

Попробуем применить TF-IDF и RandomForest - результат лучше.

In [125]:
vec = TfidfVectorizer(ngram_range=(1, 3), analyzer='char_wb', max_df=0.5, min_df=0.005)
X_train_vectorized = vec.fit_transform(X_train)
X_test_vectorized = vec.transform(X_test)

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X_train_vectorized, y_train)
preds = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds))

                 precision    recall  f1-score   support

      Abkhazian       1.00      0.87      0.93        23
         Adyghe       1.00      0.84      0.91        37
          Altai       0.99      0.98      0.98        96
         Avaric       1.00      0.90      0.95        41
        Bashkir       1.00      0.96      0.98       277
     Belarusian       1.00      0.99      0.99       627
      Bulgarian       0.94      0.99      0.96       527
         Buriat       1.00      0.87      0.93       141
        Chechen       0.90      0.96      0.93       174
        Chuvash       0.92      0.63      0.75        38
   Eastern Mari       0.87      0.97      0.92       257
          Erzya       0.92      0.87      0.89        53
         Ingush       0.88      0.44      0.59        34
      Kabardian       0.93      0.97      0.95        77
         Kalmyk       1.00      0.44      0.61        41
Karachay-Balkar       0.97      0.82      0.89       102
         Kazakh       1.00    

Grid Search пока не получается - слишком долго работает.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from tempfile import mkdtemp
from shutil import rmtree

cachedir = mkdtemp()
pipe = Pipeline(steps=[
    ('vector', CountVectorizer()),
    ('clf', RandomForestClassifier()) ],
    memory=cachedir)

param_grid = dict(
    vector=[TfidfVectorizer()],
    #clf=[LogisticRegression(max_iter=300), RandomForestClassifier()],
    vector__analyzer=['char_wb'],
    vector__max_df=[0.5, 0.75],
    vector__min_df=[0.001],
    vector__ngram_range=[(1,2)],
)
grid_search = GridSearchCV(pipe, param_grid=param_grid)
grid_search.fit(X_train, y_train)
grid_search.best_params_
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res
#preds = pipe['clf'].predict(X_test_vectorized)
#print(classification_report(y_test, preds))

In [96]:
pd.DataFrame(X_train_vectorized.toarray(), columns=vec.get_feature_names_out())

,а,аа,аб,ав,аг,ад,ае,аж,аз,ай,...,өө,өөд,өөн,өөр,ӯ,ӱ,ӱд,ӱл,ӱн,ӱр
0,0.0,0.0,0.0,0.013053,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.043198,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.124796,0.168563,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
22538,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
22539,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.161657,0.0,...,0.0,0.0,0.051285,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
22540,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


Multilayer Perceptron оказался еще лучше. 

In [126]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier().fit(X_train_vectorized, y_train)
preds = clf.predict(X_test_vectorized)
print(classification_report(y_test, preds))

res = pd.DataFrame({'txt':X_test, 'lng':y_test, 'pred':preds})
res.loc[res['lng'] != res['pred']]

                 precision    recall  f1-score   support

      Abkhazian       0.96      0.96      0.96        23
         Adyghe       1.00      1.00      1.00        37
          Altai       0.99      0.99      0.99        96
         Avaric       1.00      0.93      0.96        41
        Bashkir       0.99      0.97      0.98       277
     Belarusian       0.99      1.00      1.00       627
      Bulgarian       0.99      0.98      0.98       527
         Buriat       0.99      0.96      0.97       141
        Chechen       1.00      0.98      0.99       174
        Chuvash       0.92      0.89      0.91        38
   Eastern Mari       0.96      0.95      0.95       257
          Erzya       0.98      0.96      0.97        53
         Ingush       0.85      0.85      0.85        34
      Kabardian       1.00      0.97      0.99        77
         Kalmyk       0.94      0.80      0.87        41
Karachay-Balkar       0.86      0.91      0.89       102
         Kazakh       1.00    

,txt,lng,pred
19018,,Tatar,Serbian
677,гӏасрудал итал гӏалимчӏагӏи гӏасрудал итал гӏа...,Avaric,Ingush
10920,премия фритьофа нансена за значительные исслед...,Ossetic,Russian
5204,легальный путь прибытия на остров через аэропо...,Eastern Mari,Russian
5654,ло флаг дрк киншаса ло флаг египта каир ло фла...,Ingush,Ossetic
...,...,...,...
2589,арктагея палеагея неагея натагея,Belarusian,Tatar
10175,кипр албан ёсоор бүгд найрамдах кипр улс нь га...,Mongolian,Kalmyk
25631,къухы,Ossetic,Karachay-Balkar
22473,зоболон гасалан хоёр зургаа хоноод эртэ ерэхэ ...,Buriat,Mongolian


Ошибки все те же, либо из-за ошибок в датасете, либо слишком короткий текст, либо родственные языки. 